In [1]:
import requests
from bs4 import BeautifulSoup
import functions as fx
import os

## Get All Team Pages
> This downloads all the files from jhowell.net - each file has all seasons and outcomes of each game as well as alternate site and bowl game data

r = requests.get('https://www.jhowell.net/cf/scores/byname.htm')
soup = BeautifulSoup(r.text)
schoolLinks = soup.find_all('a') 

for school in schoolLinks:
    filename = school.get('href').split('.')[0]
    r = requests.get('https://www.jhowell.net/cf/scores/' + filename + '.htm')
    open(filename + '.htm', 'wb').write(r.content)

## Process & Insert Game Outcomes
> Each jhowell page is made up of multiple tables that each represent a season.  This code extracts data from those tables and inserts them into the GameOutcomes table.  The ID is organized by GameDate and schools are listed alphabetically to make sure that games aren't double counted.

> We don't look at the last row of each table in this script.

In [ ]:
directory = '../html/gameData/'
ext = '.htm'
for filename in os.listdir(directory):
    if (filename.endswith(ext)):
        print(filename)
        team = open(directory + filename, 'r')
        teamHTML = BeautifulSoup(team)
        seasons = teamHTML.find_all('table')
        count = 0
        allGames = []
        for season in seasons:
            ## This skips the first table that is a legend
            if (count > 0):
                games = season.find_all('tr')
                gameCount = 0
                for game in games:
                    gameData = {}
                    ## This ignores the last row of each table which is for summary
                    if (gameCount < len(games) - 1):
                        if (gameCount == 0):
                            header = (game.find('a').text).split('-')
                            school = filename.split('.')[0]
                        else:
                            gameData['Year'] = header[0]
                            gameCells = game.find_all('td')
                            cellCount = 0
                            for cell in gameCells:
                                if (cellCount == 0):
                                    gameData['GameDate'] = cell.text + '/' + header[0]
                                elif (cellCount == 1):
                                    homeAway = cell.text
                                    if (homeAway == 'vs.'):
                                        homeSchool = school
                                    elif (homeAway == '@'):
                                        awaySchool = school
                                elif (cellCount == 2):
                                    if (homeAway == 'vs.'):
                                        if (cell.find('a')):
                                            awaySchool = (cell.find('a').get('href')).split('.')[0]
                                        else:
                                            awaySchool = 'UntrackedSchool'
                                    elif (homeAway == '@'):
                                        if (cell.find('a')):
                                            homeSchool = (cell.find('a').get('href')).split('.')[0]
                                        else:
                                            homeSchool = 'UntrackedSchool'
                                elif (cellCount == 4):
                                    gameData['HomeScore'] = cell.text
                                elif (cellCount == 5):
                                    gameData['AwayScore'] = cell.text
                                elif (cellCount == 6):
                                    gameData['AlternateSite'] = 1
                                elif (cellCount == 7):
                                    gameData['BowlGame'] = 1
                                cellCount = cellCount + 1
                            if (gameData.get('AlternateSite') is None):
                                gameData['AlternateSite'] = 0   
                            if (gameData.get('BowlGame') is None):
                                gameData['BowlGame'] = 0
                            sorted_schools = sorted([homeSchool, awaySchool])
                            gameData['ID'] = gameData['GameDate'] + '_' + sorted_schools[0] + '_' + sorted_schools[1]
                            gameData['HomeSchool'] = homeSchool
                            gameData['AwaySchool'] = awaySchool

                            allGames.append(gameData)
                    gameCount = gameCount +1

            count = count + 1
        fx.toDB_gameData(allGames)

## Process & Insert Win/Loss Record Data
> This script specifically looks at the first row (to gather school name and year) and the last row of a table to gather the total win-loss-tie record.  All of that data then goes into the Records table.

In [2]:
directory = '../html/gameData/'
ext = '.htm'
for filename in os.listdir(directory):
    if (filename.endswith(ext)):
        team = open(directory + filename, 'r')
        teamHTML = BeautifulSoup(team)
        seasons = teamHTML.find_all('table')
        count = 0
        allRecords = []
        for season in seasons:
            ## This skips the first table that is a legend
            if (count > 0):
                games = season.find_all('tr')
                gameCount = 0
                for game in games:
                    recordData = {}
                    ## This ignores the last row of each table which is for summary
                    if (gameCount == 0):
                            header = (game.find('a').text).split('-')
                            school = filename.split('.')[0]
                            year = header[0]
                    if (gameCount == len(games) - 1):
                        cellCount = 0
                        gameCells = game.find_all('td')
                        cellCount = 0
                        for cell in gameCells:
                            if (cellCount == 1):
                                record = (cell.text).split('-')
                            cellCount = cellCount + 1
                        recordData['ID'] = school + '_' + year
                        recordData['Year'] = year
                        recordData['School'] = school
                        recordData['Win'] = record[0]
                        recordData['Loss'] = record[1]
                        recordData['Tie'] = record[2]
                        allRecords.append(recordData)
                    gameCount = gameCount +1
            count = count + 1
        fx.toDB_seasonData(allRecords)

Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records wr

Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 16 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 8 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records written to the database.
Completed. 22 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 0 records written to the database.
Completed. 22 records writt

## What's Next?
> You need to go year by and execute something like this:

select 
	HornbeakRating, count(*), HornbeakRating * count(*) as Sum
from SourcedPlayers as sp
	JOIN HornbeakRatings as hr
		ON sp.ID = hr.ID
where 
	sp.KeyDataSet = 5 
	and sp.College = 'maryland' 
	and sp.Year = 2015 
group by HornbeakRating

> Then you need to sum the sum column in that query as the 'RecruitRanking' value.  This could be done for Hornbeak as well as for 247 - and needs to be done for Home and Away

Create a template in queries, allowing for variables for School and Year.  Then iterate through 